# Hadi Babalou - 810199380

Artificial Intelligence - CA#01: Search Algorithms - Spring 2023
In this notebook, a food delivery problem is defined and solved using the following search algorithms:
1. Breadth First Search
2. Iterative Deepening Search
3. A* Search
4. Weighted A* Search

First, we need to define ```Graph``` class. This class is used to represent the problem map as a graph. For this purpose, first we need to define the ```Edge``` class. This class is used to represent the edges of the graph. Each edge has five instance variables:
-   ```id```: The identifier of the edge (an integer).
-   ```u``` and ```v```: The endpoints of the edge, represented as integers.
-   ```looseness```: An integer indicating the looseness of the edge. This is initially set to 0.
-   ```type```: An instance of the EdgeType enumeration, indicating whether the edge is ```REGULAR``` or ```LOOSE```. This is initially set to ```REGULAR```.

In [140]:
from enum import Enum

class EdgeType(Enum):
    REGULAR = 0
    LOOSE = 1

class Edge:
    id: int
    u: int
    v: int
    looseness: int
    # remaining_time: int
    type: EdgeType

    def __init__(self, id: int, u: int, v:int):
        self.id = id
        self.u = u
        self.v = v
        # self.remaining_time = 0
        self.looseness = 0
        self.type = EdgeType.REGULAR

    def set_as_loose(self, looseness: int) -> None:
        self.looseness = looseness
        self.type = EdgeType.LOOSE
        

Then we define the ```Node``` class. This class is used to represent a node in a graph. It has three instance variables:
-   ```id```: The identifier of the node (an integer).
-   ```type```: An instance of the NodeType enumeration, indicating the type of the node (```REGULAR```, ```ZIRAJ```, ```PIZZA``` or ```STUDENT```). This is initially set to ```REGULAR```.
-   ```edges```: A list of edges that are adjacent to the node. This list is initially empty.

In [141]:
class NodeType(Enum):
    REGULAR = 0
    ZIRAJ = 1
    PIZZA = 2
    STUDENT = 3

class Node:
    id: int
    type: NodeType
    edges: list[Edge]

    def __init__(self, id: int):
        self.id = id
        self.edges = []
        self.type = NodeType.REGULAR

    def add_edge(self, edge: Edge) -> None:
        self.edges.append(edge)

    def set_type(self, type: NodeType) -> None:
        self.type = type


Finally, we define the ```Graph``` class. This class is used to represent the problem map as a graph. It has the following instance variables:
-   ```n```: The number of nodes in the graph (an integer).
-   ```m```: The number of edges in the graph (an integer).
-   ```nodes```: A list of Node objects in the graph.
-   ```edges```: A list of Edge objects in the graph.
-   ```loose_edges```: A set of Edge objects that are marked as loose.
-   ```ziraj```: An integer indicating the node ID of the Ziraj.
-   ```students```: A dictionary that maps a student ID to a node ID.
-   ```student_demands```: A dictionary that maps a student node ID to a pizza node ID, representing the demand of each student.
-   ```students_priority```: A dictionary indicating the priority of each student over another. The Pizza of the student with the key has a higher priority than the Pizza of the student with the value.

In [142]:
class Graph:
    n: int
    m: int
    nodes: list[Node]
    edges: list[Edge]
    loose_edges: set[Edge]
    ziraj: int
    # pizzas: set[int]
    students: dict[int, int]            # student_id: node_id
    student_demands: dict[int, int]     # student_node: pizza_node
    students_priority: dict[int, int]   # student1_node: student2_node

    def __init__(self, n: int, m: int):
        self.n = n
        self.m = m
        self.nodes = [Node(i) for i in range(0, n)]
        self.edges = []
        self.loose_edges = set()
        self.ziraj = -1
        self.pizzas = set()
        self.students = dict()
        self.student_demands = {}
        self.students_priority = {}

    def __str__(self) -> str:
        _str_ = ""
        _str_ += f"n: {self.n}, m: {self.m}"
        
        return _str_

    
    def add_edge(self, edge: Edge) -> None:
        self.edges.append(edge)
        self.nodes[edge.u].add_edge(edge)
        self.nodes[edge.v].add_edge(edge)

    def set_loose_edge(self, edge_id: int, looseness: int) -> None:
        edge = self.edges[edge_id]
        edge.set_as_loose(looseness)
        self.loose_edges.add(edge)

    def set_ziraj(self, node_id: int) -> None:
        self.ziraj = node_id
        self.nodes[node_id].set_type(NodeType.ZIRAJ)

    def add_demand(self, student_id: int, student_node:int, pizza_node: int) -> None:
        self.students[student_id] = student_node
        self.nodes[student_node].set_type(NodeType.STUDENT)
        self.nodes[pizza_node].set_type(NodeType.PIZZA)
        self.student_demands[student_node] = pizza_node

    def add_priority(self, student1_id: int, student2_id: int) -> None:
        self.students_priority[self.students[student1_id]] = self.students[student2_id]


The State class is used to represent a state in the search space. It has the following instance variables:
-   ```agent_loc```: The location of the agent (an integer).
-   ```done_students```: A set of student locations whose demands have already been satisfied by the agent.
-   ```current_pickup```: The Node ID of the pizzaria that the agent is currently carrying a pizza from. This is initially set to None.
-   ```loose_status```: A dictionary that maps an ```Edge``` object to the remaining time during which the edge can be traversed. This dictionary represents the looseness status of the edges.
-   ```path_so_far```: A list of node IDs that the agent has visited so far.
-   ```time_on_node```: The time spent by the agent at the current node (an integer).

In [143]:
class State:
    agent_loc: int
    done_pizzas: set[int]
    done_students: set[int]
    current_pickup: int
    loose_status: dict[int, int]
    path_so_far: list[int]
    time_on_node: int

    def __init__(self, 
                agent_loc: int, 
                done_pizzas: set[int], 
                done_students: set[int], 
                current_pickup: int,
                loose_status: dict[int, int],
                path_so_far: list[int],
                time_on_node: int, 
                ):
        self.agent_loc = agent_loc
        self.done_pizzas = done_pizzas
        self.done_students = done_students
        self.current_pickup = current_pickup
        self.loose_status = loose_status
        self.path_so_far = path_so_far
        self.time_on_node = time_on_node
        
    def __eq__(self, other: object) -> bool:
        return isinstance(other, State) and (
                self.agent_loc == other.agent_loc and 
                self.done_pizzas == other.done_pizzas and 
                self.done_students == other.done_students and 
                self.current_pickup == other.current_pickup and 
                self.time_on_node == other.time_on_node # and 
                # self.loose_status == other.loose_status and
                # self.path_so_far == other.path_so_far
            )
    
    def __str__(self) -> str:
        # loose status: {edge, remaining_time for edge, remaining_time in self.loose_status.items()}, \
        _str_ = ""
        _str_ += f"agent loc: {self.agent_loc}, "
        _str_ += f"path so far: {self.path_so_far}, "
        _str_ += f"current pickup: {self.current_pickup}, "
        _str_ += f"time on node: {self.time_on_node}, "
        _str_ += f"done pizzas: {self.done_pizzas}, "
        _str_ += f"done students: {self.done_students}, "
        return _str_

    def __hash__(self) -> int:
        _str_ = ""
        _str_ += f"{self.agent_loc}, "
        _str_ += f"{self.done_pizzas}, "
        _str_ += f"{self.done_students}, "
        _str_ += f"{self.current_pickup}, "
        _str_ += f"{self.time_on_node}, "
        return hash(_str_)

    def relax(self) -> None:
        for edge, remaining_time in self.loose_status.items():
            if remaining_time > 0:
                self.loose_status[edge] = remaining_time - 1


The ```Search``` class contains an instance of the ```Graph``` class and some methods that are used to solve the problem. 
-   ```is_goal``` method is used to check whether a given state is a goal state or not.
-   ```init_state``` method is used to initialize the initial state of the problem.
-   ```can_pickup``` method is used to check whether the agent can pickup a pizza from a given pizzaria or not (based on students priority).
-   ```next_states``` method is used to generate the next states from a given state. This method is used by the search algorithms to expand the search space. 

In [144]:
from copy import deepcopy

class Search:
    graph: Graph

    def __init__(self, graph: Graph):
        self.graph = graph

    def is_goal(self, state: State) -> bool:
        # print ("-----", len(state.done_pizzas), "!= ", len(self.graph.pizzas), end=" ")
        # print (len(state.done_students), "!= ", len(self.graph.students), "-----")
        return (len(state.done_students) == len(self.graph.students))

    def init_state(self) -> State:
        _loose_status_ = {}
        for edge in self.graph.loose_edges:
            _loose_status_[edge.id] = 0

        return State(
            agent_loc=self.graph.ziraj,
            done_pizzas=set(),
            done_students=set(),
            current_pickup=-1,
            loose_status=_loose_status_,
            path_so_far=[self.graph.ziraj],
            time_on_node=0,
        )
    
    def can_pickup(self, state: State, pizza: int) -> bool:
        # check if we can pickup pizza based on state's students_priority
        demanding_student = -1
        for key, value in self.graph.student_demands.items():
            if value == pizza:
                demanding_student = key
        if demanding_student == -1:
            return False
        
        for high_priority, low_priority in self.graph.students_priority.items():
            if demanding_student == low_priority:
                if high_priority not in state.done_students:
                    return False

        return True

    def next_states(self, cur_state: State) -> list[State]:
        next_states = []
        current_node = self.graph.nodes[cur_state.agent_loc]

        # go to adjacent nodes
        for edge in current_node.edges:
            new_state = deepcopy(cur_state)
            next_node = deepcopy(current_node)

            if edge.v == current_node.id:
                next_node = self.graph.nodes[edge.u]
            elif edge.u == current_node.id:
                next_node = self.graph.nodes[edge.v]

            if edge.type == EdgeType.LOOSE:                    
                if cur_state.loose_status[edge.id] > 0:
                    # stay on current node
                    new_state = deepcopy(cur_state)
                    new_state.path_so_far.append(current_node.id)
                    new_state.time_on_node += 1
                    new_state.relax()
                    next_states.append(new_state)
                    continue
                else:
                    new_state.loose_status[edge.id] = edge.looseness + 1

            new_state.agent_loc = next_node.id
            new_state.path_so_far.append(next_node.id)
            new_state.time_on_node = 0

            if next_node.type == NodeType.STUDENT:
                # print("===== 1:", next_node.id, "=====")
                if next_node.id not in new_state.done_students:
                    # print("===== 2 =====")
                    if new_state.current_pickup == self.graph.student_demands[next_node.id]:
                        # print("===== 3 =====")
                        new_state.done_students.add(next_node.id)
                        new_state.done_pizzas.add(new_state.current_pickup)
                        new_state.current_pickup = -1
            
            elif next_node.type == NodeType.PIZZA:
                if next_node.id not in new_state.done_pizzas:
                    if (self.can_pickup(new_state, next_node.id)):
                        # i can pickup pizza here or leave it. both of this states should be added to next_states
                        # this handles picking up pizza. leaving it will be handled outside of this scope
                        new_state2 = deepcopy(new_state)
                        new_state2.current_pickup = next_node.id
                        new_state2.relax()
                        next_states.append(new_state2)

            new_state.relax()
            next_states.append(new_state)
            
        return next_states
    

Now we define the ```BFS``` function. This function is used to solve the problem using the Breadth First Search algorithm. It takes an instance of the ```Search``` class as input and returns a list of node IDs that the agent should visit to deliver all pizzas. If there is no solution, it returns ```None```. 

The function initializes the initial state and adds it to a frontier list, which initially contains only the initial state. It also initializes an explored set to keep track of states that have been already explored.

The function then enters into a loop which continues until the frontier list is non-empty. In each iteration of the loop, the function removes the first state from the frontier list and adds it to the explored set, indicating that it has been visited. The function then generates all the possible next states that can be reached from the current state using the ```next_states``` method of the Search object, and adds only those states which have not been already explored or added to the frontier list.

If the current state is a goal state, the function returns the path from the initial state to the goal state. If no goal state is found, the function returns an empty list.

In [145]:
def BFS(problem: Search) -> tuple[list[int], int]:

    init_state = problem.init_state()
    if problem.is_goal(init_state):
        # TODO
        return init_state.path_so_far, 0

    explored = set()
    frontier = [init_state]
    states_expanded = 0

    while frontier:
        state = frontier.pop(0)
        explored.add(state)
        states_expanded += 1
        # print("expanded=", states_expanded, "state=", state.__str__())

        if problem.is_goal(state):
            # print("states expanded:", states_expanded)
            return state.path_so_far, states_expanded

        for next_state in problem.next_states(state):
            if next_state not in explored and next_state not in frontier:
                frontier.append(next_state)
                # print(":::", next_state.__str__(), ":::")

    # print("states expanded:", states_expanded)
    return [], states_expanded



The ```get_input``` function is used to read the input from the input file and create an instance of the ```Search``` class. It takes the name of the input file as input and returns an instance of the ```Search``` class.

In [146]:
def get_input(testcase_path: str) -> Search:
    file = open(testcase_path, "r")
    
    n, m = map(int, file.readline().split())
    graph = Graph(n, m)

    for i in range(m):
        u, v = map(int, file.readline().split())
        edge = Edge(i, u-1, v-1)
        graph.add_edge(edge)

    h = int(file.readline())
    for _ in range(h):
        i, xi = map(int, file.readline().split())
        graph.set_loose_edge(i-1, xi)

    v = int(file.readline())
    graph.set_ziraj(v-1)

    s = int(file.readline())
    for student_id in range(1, s+1):
        p, q = map(int, file.readline().split())
        graph.add_demand(student_id, p-1, q-1)
        
    t = int(file.readline())
    for i in range(1, t+1):
        a, b = map(int, file.readline().split())
        graph.add_priority(a, b)

    file.close()
    return Search(graph)


In [147]:
search = get_input("testcase/3.txt")
path, nodes_expanded = BFS(search)
print("nodes expanded:", nodes_expanded)
print("path cost:", len(path))
for i in range (len(path)):
    node = path[i]
    if i != len(path) - 1:
        print(node + 1,"->", end=" ")
    else:
        print(node + 1)

nodes expanded: 840
path cost: 19
11 -> 6 -> 5 -> 10 -> 7 -> 10 -> 5 -> 1 -> 2 -> 9 -> 14 -> 6 -> 8 -> 3 -> 1 -> 1 -> 3 -> 7 -> 13


In [148]:
import time
from typing import Callable

TESTCASES_PATH = "testcase/"
TESTS_REPEAT = 3
TESTS_COUNT = 7

def run(algorithm: Callable[[Search], tuple[list[int], int]]) -> None:
    for i in range(TESTS_COUNT):
        testcase_path = TESTCASES_PATH + str(i+1) + ".txt"
        search = get_input(testcase_path)
        duration = 0
        path = []
        nodes_expanded = 0
        for j in range(TESTS_REPEAT):
            start = time.time()
            path, nodes_expanded = algorithm(search)
            end = time.time()
            duration += end - start
        duration /= TESTS_REPEAT

        print("testcase", i+1)
        if len(path) == 0:
            print("no path found")
        
        print("path cost:", len(path))
        for j in range (len(path)):
            if (j == len(path) - 1):
                print(path[j] + 1)
                break
            print(path[j] + 1,"->", end=" ")
    
        print("nodes expanded:", nodes_expanded)
        print("duration:", duration)
        if i != (TESTS_COUNT - 1):
            print("---------------------------------")
        

In [149]:
print("Testing BFS algorithm:")
print()
run(BFS)
print()
print("================================================================================")
print()


Testing BFS algorithm:

testcase 1
path cost: 12
5 -> 7 -> 2 -> 9 -> 4 -> 8 -> 10 -> 3 -> 8 -> 1 -> 6 -> 2
nodes expanded: 284
duration: 0.23342108726501465
---------------------------------
testcase 2
path cost: 17
2 -> 14 -> 11 -> 9 -> 15 -> 6 -> 9 -> 14 -> 1 -> 5 -> 8 -> 10 -> 6 -> 4 -> 6 -> 9 -> 7
nodes expanded: 1381
duration: 0.9999454816182455
---------------------------------
testcase 3
path cost: 19
11 -> 6 -> 5 -> 10 -> 7 -> 10 -> 5 -> 1 -> 2 -> 9 -> 14 -> 6 -> 8 -> 3 -> 1 -> 1 -> 3 -> 7 -> 13
nodes expanded: 840
duration: 0.415679931640625
---------------------------------
testcase 4
path cost: 10
1 -> 2 -> 3 -> 4 -> 4 -> 4 -> 3 -> 2 -> 3 -> 5
nodes expanded: 25
duration: 0.005681832631429036
---------------------------------
testcase 5
path cost: 11
6 -> 4 -> 10 -> 5 -> 3 -> 8 -> 9 -> 1 -> 7 -> 9 -> 2
nodes expanded: 351
duration: 0.24752060572306314
---------------------------------
testcase 6
path cost: 20
14 -> 1 -> 14 -> 12 -> 3 -> 9 -> 4 -> 15 -> 12 -> 3 -> 12 -> 6 -> 